# Environment setup

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install imageio
# !pip install git+https://github.com/tensorflow/docs

In [ ]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pandas as pd

from PIL import Image, ImageOps
from tensorflow.keras import layers
import time
# import tensorflow_docs.vis.embed as embed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib.pyplot import imshow

from IPython import display

tf.config.run_functions_eagerly(False)

In [ ]:
# Use the GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Define GAN Architecture

In [ ]:
def make_generator_model(noise_dim=40, data_dim=42):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, use_bias=False, input_shape=(noise_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    assert model.output_shape == (None, 128)  # Note: None is the batch size

    model.add(layers.Dense(64, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    assert model.output_shape == (None, 64)

    model.add(layers.Dense(32, use_bias=True))
    model.add(layers.Dropout(0.2))
    model.add(layers.LeakyReLU())

    assert model.output_shape == (None, 32)

    model.add(layers.Dense(data_dim-1, use_bias=False))
    # model.add(layers.LeakyReLU())

    assert model.output_shape == (None, data_dim - 1)

    # print(model.summary())

    return model

def make_discriminator_model(data_dim=42):  
    model = tf.keras.Sequential()
    model.add(layers.Dense(32, input_shape=(data_dim,)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(16))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


def discriminator_loss(real_output, fake_output, cross_entropy, d_loss):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    # d_loss.append(fake_loss)
    total_loss = real_loss + fake_loss
    return total_loss


def generator_loss(fake_output, cross_entropy):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images, generator_optimizer0, generator_optimizer1, discriminator_optimizer, cross_entropy, noise_dim, d_hist, g_hist, g_hist1, d_loss):
    noise = tf.random.normal([images.get_shape()[0], noise_dim], dtype=tf.dtypes.float64)
    # noise = tf.cast(tf.concat([noise, tf.round(tf.reshape(tf.gather(images, [images.get_shape()[1]-1], axis=1), [images.get_shape()[0], 1]))], 1), tf.dtypes.float64)

    # print(noise.shape)

    with tf.GradientTape() as gen_tape0, tf.GradientTape() as gen_tape1, tf.GradientTape() as disc_tape:
        generated_images0 = generator0(noise, training=False)
        # generated_images0 = tf.concat([tf.cast(generated_images0, tf.dtypes.float64) , tf.round(tf.reshape(tf.gather(images, [images.get_shape()[1]-1], axis=1), [images.get_shape()[0], 1]))], 1)
        generated_images1 = generator1(noise, training=False)
        generated_images0 = tf.concat([generated_images0, tf.zeros([images.get_shape()[0], 1])], 1)
        generated_images1 = tf.concat([generated_images1, tf.ones([images.get_shape()[0], 1])], 1)

        generated_images = tf.concat([generated_images0, generated_images1], 0)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        disc_loss = discriminator_loss(real_output, fake_output, cross_entropy, d_loss)

        generated_images0 = generator0(noise, training=True)
        generated_images1 = generator1(noise, training=True)
        generated_images0 = tf.concat([generated_images0, tf.zeros([images.get_shape()[0], 1])], 1)
        generated_images1 = tf.concat([generated_images1, tf.ones([images.get_shape()[0], 1])], 1)

        real_output = discriminator(images, training=False)
        fake_output0 = discriminator(generated_images0, training=False)
        fake_output1 = discriminator(generated_images1, training=False)

        gen_loss0 = generator_loss(fake_output0, cross_entropy)
        gen_loss1 = generator_loss(fake_output1, cross_entropy)

        # g_hist.append(gen_loss0.numpy())
        # g_hist1.append(gen_loss1.numpy())
        # d_hist.append(disc_loss.numpy())

    gradients_of_generator0 = gen_tape0.gradient(gen_loss0, generator0.trainable_variables)
    gradients_of_generator1 = gen_tape1.gradient(gen_loss1, generator1.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer0.apply_gradients(zip(gradients_of_generator0, generator0.trainable_variables))
    generator_optimizer1.apply_gradients(zip(gradients_of_generator1, generator1.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs, generator_optimizer0, generator_optimizer1, discriminator_optimizer, cross_entropy, noise_dim):
    # plt.figure()
    for epoch in range(epochs):
      start = time.time()

      d_hist, g_hist, g_hist1, d_loss = list(), list(), list(), list()
      for image_batch in dataset:
        train_step(image_batch, generator_optimizer0, generator_optimizer1, discriminator_optimizer, cross_entropy, noise_dim, d_hist, g_hist, g_hist1, d_loss)

      # d_hist = np.mean(d_hist)
      # g_hist = np.mean(g_hist)
      # g_hist1 = np.mean(g_hist1)
      # d_loss = np.mean(d_loss)
      # print(g_hist)
      # print(g_hist1)
      # print(d_hist)
      # print(d_loss)
      # plot_history(d_hist, g_hist, epoch)

      print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

def plot_history(d_hist, g_hist, x):
	# plot loss
	plt.plot(x, d_hist, label='d')
	plt.plot(x, g_hist, label='g')
	plt.legend()

# Data

In [ ]:
### DEFINE VARIABLES ###
global BUFFER_SIZE
global BATCH_SIZE
global EPOCHS

BUFFER_SIZE = 1100
BATCH_SIZE = 1024
EPOCHS = 1000

path_train = 'Train.csv'

In [ ]:
### LOAD DATA ###
X = pd.read_csv(path_train)
X

,protocol_type,service,flag,src_bytes,dst_bytes,logged_in,count,srv_count,same_srv_rate,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,attack
0,1,29,1,-1.028654,-0.842833,0,1.156899,-0.938302,0,2,1.000000,0.007843,0.01,0.07,0.00,1
1,2,48,9,0.090010,-0.842833,0,0.744279,1.843540,2,0,1.000000,0.368627,0.37,0.62,0.37,1
2,2,12,9,0.235974,0.523887,0,1.083690,2.362526,2,0,0.411765,0.258824,0.63,0.03,0.01,0
3,1,6,5,-1.028654,-0.842833,0,0.897136,-0.053807,1,1,1.000000,0.003922,0.00,0.07,0.00,1
4,1,24,9,0.851508,1.128924,1,-1.075020,-0.706689,2,0,1.000000,1.000000,1.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92134,1,24,9,0.842116,1.844348,1,-0.710057,-0.256141,2,0,0.439216,1.000000,1.00,0.00,0.01,0
92135,1,24,9,0.865116,2.128232,1,-1.288509,-0.938302,2,0,1.000000,1.000000,1.00,0.00,0.00,0
92136,1,20,9,2.353774,-0.842833,1,-1.075020,-0.938302,2,0,0.039216,0.309804,0.80,0.30,0.80,0
92137,1,24,9,0.760153,1.044703,1,-0.558584,-0.148634,2,0,0.027451,1.000000,1.00,0.00,0.14,0


In [ ]:
### Categorical ranges ###
cat_columns = ["protocol_type", "service", "flag", "same_srv_rate", "diff_srv_rate", "logged_in"]
cat_codes = {}
for col in cat_columns:
  cat_codes[col] = X[col].unique()

df_X = X

In [ ]:
cat_codes

{'diff_srv_rate': array([2, 0, 1]),
 'flag': array([ 1,  9,  5,  2,  4,  3,  0,  7,  8, 10,  6]),
 'logged_in': array([0, 1]),
 'protocol_type': array([1, 2, 0]),
 'same_srv_rate': array([0, 2, 1]),
 'service': array([29, 48, 12,  6, 24, 34, 62, 14, 20, 11, 52, 35,  8, 15, 43, 58, 13,
        65, 36, 41,  5, 32, 17, 37, 30, 60, 53, 39, 18,  2, 57, 33, 16, 23,
        19, 28, 40,  7, 21, 56,  4, 46, 63, 10, 42, 31, 55, 49, 54,  9, 64,
        26, 51, 38, 50, 47, 45, 22, 61, 44, 27,  1, 25,  3,  0, 59])}

In [ ]:
### Add noise to categorical columns ###
noise = np.random.normal(0,0.1,(len(X),len(cat_columns)))
noise[noise > 0.2] = 0.2
noise[noise < -0.2] = -0.2
X[cat_columns] = X[cat_columns] + noise

# for col in cat_columns + []:
#   X[col] = X[col] / np.max(cat_codes[col])

df_X = X
df_X

,protocol_type,service,flag,src_bytes,dst_bytes,logged_in,count,srv_count,same_srv_rate,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,attack
0,0.804169,29.066497,0.924940,-1.028654,-0.842833,-0.003966,1.156899,-0.938302,0.004003,1.871793,1.000000,0.007843,0.01,0.07,0.00,1
1,1.800000,48.007992,9.038486,0.090010,-0.842833,-0.155811,0.744279,1.843540,2.120952,0.006978,1.000000,0.368627,0.37,0.62,0.37,1
2,1.988411,11.919183,9.200000,0.235974,0.523887,0.103321,1.083690,2.362526,1.925570,-0.145919,0.411765,0.258824,0.63,0.03,0.01,0
3,0.929805,5.978107,5.055023,-1.028654,-0.842833,-0.055295,0.897136,-0.053807,0.968579,0.945777,1.000000,0.003922,0.00,0.07,0.00,1
4,1.080933,23.822888,9.200000,0.851508,1.128924,1.117515,-1.075020,-0.706689,1.935833,-0.127341,1.000000,1.000000,1.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92134,1.031245,23.921560,9.155876,0.842116,1.844348,0.928427,-0.710057,-0.256141,1.867392,-0.080218,0.439216,1.000000,1.00,0.00,0.01,0
92135,1.086161,23.898786,9.035107,0.865116,2.128232,1.146370,-1.288509,-0.938302,2.058883,-0.068138,1.000000,1.000000,1.00,0.00,0.00,0
92136,0.946292,19.835558,8.996095,2.353774,-0.842833,0.893624,-1.075020,-0.938302,1.883830,0.038070,0.039216,0.309804,0.80,0.30,0.80,0
92137,0.955492,23.989873,9.127267,0.760153,1.044703,1.060554,-0.558584,-0.148634,2.033807,-0.122036,0.027451,1.000000,1.00,0.00,0.14,0


In [ ]:
### Vectorice to numpy ###
columns = X.columns
X = X.to_numpy()
X

array([[8.04169295e-01, 2.90664965e+01, 9.24939865e-01, ...,
        7.00000000e-02, 0.00000000e+00, 1.00000000e+00],
       [1.80000000e+00, 4.80079924e+01, 9.03848647e+00, ...,
        6.20000000e-01, 3.70000000e-01, 1.00000000e+00],
       [1.98841112e+00, 1.19191829e+01, 9.20000000e+00, ...,
        3.00000000e-02, 1.00000000e-02, 0.00000000e+00],
       ...,
       [9.46291930e-01, 1.98355576e+01, 8.99609523e+00, ...,
        3.00000000e-01, 8.00000000e-01, 0.00000000e+00],
       [9.55491792e-01, 2.39898732e+01, 9.12726732e+00, ...,
        0.00000000e+00, 1.40000000e-01, 0.00000000e+00],
       [9.78966933e-01, 2.42000000e+01, 8.95564465e+00, ...,
        0.00000000e+00, 2.00000000e-02, 0.00000000e+00]])

In [ ]:
### RESHAPE DATA ###
train_data = X.reshape(X.shape[0], X.shape[1])

In [ ]:
noise_dim = 8
data_dim = X.shape[1]
data_dim

16

# Build and Run

In [ ]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

generator0 = make_generator_model(noise_dim, data_dim)
generator1 = make_generator_model(noise_dim, data_dim)

# noise = tf.random.normal([1, noise_dim])
# noise = tf.concat([noise, tf.ones([1,1])], 1)
# generated_malware = generator(noise, training=False)
# generated_malware = tf.concat([generated_malware, tf.ones([1,1])], 1)

discriminator = make_discriminator_model(data_dim)
# decision = discriminator(generated_malware)
# print(decision)

# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

generator_optimizer0 = tf.keras.optimizers.Adam(learning_rate=0.001)
generator_optimizer1 = tf.keras.optimizers.Adam(learning_rate=0.001)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

train(train_dataset, EPOCHS, generator_optimizer0, generator_optimizer1, discriminator_optimizer, cross_entropy, noise_dim)

Time for epoch 1 is 6.640403509140015 sec
Time for epoch 2 is 0.6644165515899658 sec
Time for epoch 3 is 0.6546406745910645 sec
Time for epoch 4 is 0.6138420104980469 sec
Time for epoch 5 is 0.6492741107940674 sec
Time for epoch 6 is 0.6179847717285156 sec
Time for epoch 7 is 0.6574780941009521 sec
Time for epoch 8 is 0.6471512317657471 sec
Time for epoch 9 is 0.6092479228973389 sec
Time for epoch 10 is 0.6359846591949463 sec
Time for epoch 11 is 0.6531720161437988 sec
Time for epoch 12 is 0.6507508754730225 sec
Time for epoch 13 is 1.2708446979522705 sec
Time for epoch 14 is 0.6069519519805908 sec
Time for epoch 15 is 0.6804418563842773 sec
Time for epoch 16 is 0.6177325248718262 sec
Time for epoch 17 is 0.615532636642456 sec
Time for epoch 18 is 0.629202127456665 sec
Time for epoch 19 is 0.6040501594543457 sec
Time for epoch 20 is 0.6001641750335693 sec
Time for epoch 21 is 0.6325931549072266 sec
Time for epoch 22 is 0.622492790222168 sec
Time for epoch 23 is 0.6326923370361328 sec
T

# Generate and Discriminate

In [ ]:
num_samples = 100000
num_samples_1 = int(num_samples/2)
num_samples_0 = int(num_samples - num_samples_1)

noise = tf.random.normal([num_samples_0, noise_dim])
labels = tf.concat([tf.ones([num_samples_1,1]), tf.zeros([num_samples_0,1])], 0)
# noise = tf.concat([noise, labels], 1)
# generated_malware = generator(noise, training=False)
generated_malware0 = generator0(noise, training=False)

noise = tf.random.normal([num_samples_1, noise_dim])
generated_malware1 = generator1(noise, training=False)

generated_malware = tf.concat([generated_malware1, generated_malware0], 0)
generated_malware = tf.concat([generated_malware, labels], 1)
generated_malware

<tf.Tensor: shape=(100000, 16), dtype=float32, numpy=
array([[-1.56389356e-01,  5.00511742e+00,  4.98795986e+00, ...,
        -2.15969354e-01,  5.72412670e-01,  1.00000000e+00],
       [ 1.39453709e+00,  7.22771454e+01,  6.41177082e+00, ...,
         6.01485521e-02, -7.91153789e-01,  1.00000000e+00],
       [ 7.33363092e-01,  4.15472870e+01,  4.00114298e+00, ...,
         1.29937306e-01, -9.68943723e-03,  1.00000000e+00],
       ...,
       [ 9.80542898e-01,  3.15154381e+01,  9.92795181e+00, ...,
         1.90067351e-01, -4.28379059e-01,  0.00000000e+00],
       [ 9.19324338e-01,  2.89860439e+01,  9.91329098e+00, ...,
         4.82860431e-02, -2.96291083e-01,  0.00000000e+00],
       [ 8.43213916e-01,  3.15436592e+01,  1.04393635e+01, ...,
        -2.53585409e-02, -2.99624443e-01,  0.00000000e+00]], dtype=float32)>

In [ ]:
decision = discriminator(generated_malware)
decision

<tf.Tensor: shape=(100000, 1), dtype=float32, numpy=
array([[ 0.17005745],
       [ 0.33248356],
       [ 0.17794168],
       ...,
       [-0.1603883 ],
       [-0.12531012],
       [-0.13347799]], dtype=float32)>

# Helper: Round Categorical

In [ ]:
def round_categorical(df_samples, cat_columns=[], cat_codes={}):
  # for col in cat_columns:
  #   df_samples[col] = df_samples[col] * (len(cat_codes[col].keys())-1) 

  # for col in cat_columns:
  #   df_samples[col] = df_samples[col] * np.max(cat_codes[col])
  
  df_samples[cat_columns] = np.around(df_samples[cat_columns])
  df_samples[df_samples[cat_columns] <= 0] = 0

  # If a value is bigger than the largest category code, map it to largest category code
  for col in cat_columns:
    largest_code_value = max(k for k in cat_codes[col])
    df_samples.loc[df_samples[col] > largest_code_value, col] = largest_code_value
    # df_samples[col] = df_samples[col] / (len(cat_codes[col].keys())-1)
  
  df_samples.loc[df_samples['dst_host_count'] <= 0, 'dst_host_count'] = 0
  df_samples.loc[df_samples['dst_host_count'] > 1, 'dst_host_count'] = 1

  df_samples['dst_host_same_srv_rate'] = np.around(df_samples['dst_host_same_srv_rate'], 2)
  df_samples.loc[df_samples['dst_host_same_srv_rate'] <= 0, 'dst_host_same_srv_rate'] = 0
  df_samples.loc[df_samples['dst_host_same_srv_rate'] > 1, 'dst_host_same_srv_rate'] = 1

  df_samples['dst_host_diff_srv_rate'] = np.around(df_samples['dst_host_diff_srv_rate'], 2)
  df_samples.loc[df_samples['dst_host_diff_srv_rate'] <= 0, 'dst_host_diff_srv_rate'] = 0
  df_samples.loc[df_samples['dst_host_diff_srv_rate'] > 1, 'dst_host_diff_srv_rate'] = 1

  df_samples['dst_host_same_src_port_rate'] = np.around(df_samples['dst_host_same_src_port_rate'], 2)
  df_samples.loc[df_samples['dst_host_same_src_port_rate'] <= 0, 'dst_host_same_src_port_rate'] = 0
  df_samples.loc[df_samples['dst_host_same_src_port_rate'] > 1, 'dst_host_same_src_port_rate'] = 1

  df_samples['dst_host_diff_srv_rate'] = np.around(df_samples['dst_host_diff_srv_rate'], 2)
  df_samples.loc[df_samples['dst_host_diff_srv_rate'] <= 0, 'dst_host_diff_srv_rate'] = 0
  df_samples.loc[df_samples['dst_host_diff_srv_rate'] > 1, 'dst_host_diff_srv_rate'] = 1

  return df_samples

In [ ]:
np_generated_malware = generated_malware.numpy()
df_generated_malware = pd.DataFrame(np_generated_malware, columns=columns)
df_generated_malware

,protocol_type,service,flag,src_bytes,dst_bytes,logged_in,count,srv_count,same_srv_rate,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,attack
0,-0.156389,5.005117,4.987960,-0.762965,-1.784852,0.423991,0.801866,1.268683,0.598050,0.541843,0.194191,0.225932,0.581499,-0.215969,0.572413,1.0
1,1.394537,72.277145,6.411771,-2.340615,0.060235,0.631315,1.521578,-1.848490,0.616104,1.914891,1.767916,0.401121,0.449261,0.060149,-0.791154,1.0
2,0.733363,41.547287,4.001143,-1.580013,-1.106966,0.149538,0.961440,-0.955936,0.509365,1.126811,1.074611,0.059435,0.344914,0.129937,-0.009689,1.0
3,0.672015,34.323189,4.890115,-1.632406,-1.093457,0.366096,1.237212,0.090182,0.552009,1.252963,1.083295,0.323284,0.518495,-0.114887,-0.049139,1.0
4,0.415534,27.253134,4.201684,-1.416219,-1.416771,0.230661,1.017064,-0.094760,0.428460,0.987871,0.811351,0.106744,0.425604,-0.028253,0.152595,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.988113,31.984766,10.052040,0.460960,1.406243,1.130925,-1.105775,0.244184,2.791420,-0.033143,0.280456,0.956300,0.928741,0.188104,-0.422227,0.0
99996,0.995580,30.617098,9.906360,0.441352,1.306343,1.092389,-1.120778,0.165066,2.710434,0.016676,0.306990,0.917081,0.929628,0.173654,-0.377182,0.0
99997,0.980543,31.515438,9.927952,0.488869,1.372716,1.097096,-1.079590,0.237897,2.762645,-0.033406,0.294639,0.937539,0.906862,0.190067,-0.428379,0.0
99998,0.919324,28.986044,9.913291,0.407447,1.061709,1.051663,-1.142364,0.352029,2.639138,-0.054686,0.414866,0.881907,0.819689,0.048286,-0.296291,0.0


In [ ]:
generated_malware_rounded = round_categorical(df_generated_malware, cat_columns, cat_codes=cat_codes)
generated_malware_rounded

,protocol_type,service,flag,src_bytes,dst_bytes,logged_in,count,srv_count,same_srv_rate,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,attack
0,0.0,5.0,5.0,-0.762965,-1.784852,0.0,0.801866,1.268683,1.0,1.0,0.194191,0.225932,0.58,0.00,0.57,1.0
1,1.0,65.0,6.0,-2.340615,0.060235,1.0,1.521578,-1.848490,1.0,2.0,1.000000,0.401121,0.45,0.06,0.00,1.0
2,1.0,42.0,4.0,-1.580013,-1.106966,0.0,0.961440,-0.955936,1.0,1.0,1.000000,0.059435,0.34,0.13,0.00,1.0
3,1.0,34.0,5.0,-1.632406,-1.093457,0.0,1.237212,0.090182,1.0,1.0,1.000000,0.323284,0.52,0.00,0.00,1.0
4,0.0,27.0,4.0,-1.416219,-1.416771,0.0,1.017064,-0.094760,0.0,1.0,0.811351,0.106744,0.43,0.00,0.15,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.0,32.0,10.0,0.460960,1.406243,1.0,-1.105775,0.244184,2.0,0.0,0.280456,0.956300,0.93,0.19,0.00,0.0
99996,1.0,31.0,10.0,0.441352,1.306343,1.0,-1.120778,0.165066,2.0,0.0,0.306990,0.917081,0.93,0.17,0.00,0.0
99997,1.0,32.0,10.0,0.488869,1.372716,1.0,-1.079590,0.237897,2.0,0.0,0.294639,0.937539,0.91,0.19,0.00,0.0
99998,1.0,29.0,10.0,0.407447,1.061709,1.0,-1.142364,0.352029,2.0,0.0,0.414866,0.881907,0.82,0.05,0.00,0.0


In [ ]:
decision = discriminator(generated_malware_rounded.to_numpy())
decision

<tf.Tensor: shape=(100000, 1), dtype=float32, numpy=
array([[ 0.11941531],
       [ 0.52604043],
       [ 0.20191905],
       ...,
       [-0.1387291 ],
       [-0.11991832],
       [-0.204688  ]], dtype=float32)>

In [ ]:
generated_malware_rounded.to_csv('generated.csv', index=False)